------

## The Partial Sum Problem

Burton Rosenberg

29 May 2023

------



### The partial sum problem

The problem is to find for each $i \in [0,n)$ the values $\sum_{0\le j\le i } a_i$. Serially, it is necessary to touch each location in order therefore $O(n)$ is optimal, and is also obvious. 

In [1]:
import numpy as np

def partial_sum_cpu(a):
    for i in range(1,len(a)):
        a[i] += a[i-1]
    
    
def test_partial_sum_cpu(n):
    a = np.random.randint(0,n,n)
    print(a)
    partial_sum_cpu(a)
    print(a)
    assert (sum(a)!=a[-1]), "checking that one value is correct"
    
 
test_partial_sum_cpu(10)

[7 6 1 7 9 4 7 4 5 7]
[ 7 13 14 21 30 34 41 45 50 57]


On a GPU this is solvable by a $O(\log n)$ depth (iterations) using $O(n)$ processors. Not that the time-space product is $O(n \log n)$ in parallel and only $O(n)$ in serial. We trade off time for space with a slight cost overall.

### Block method

One idea is based on the following observation. Let $s_{i,j} = \sum_{k\in [i,j)} a_k$. Then for a $i<k<j$ we have
$s_{i,j} = s_{i,k}+s_{k,j}$.

Work in blocks of size $2^k$ for passes enumerated by $k$ starting at $k=0$. The _loop invariant_ is that each $2^k$ sized block, aligned with starting index $i\cdot 2^k$ correctly as the paritial sum for that block, 
$
s_{i\cdot 2^k, (i+1)\cdot 2^k}
$
The proceed from $2^k$ to $2^{k+1}$ but adding to the "top block" the sum of all the numbers in the bottom block.

<pre>
L.I. a[j] has the partial sum up to i within the block i 2^k &le j < (i+1) 2^(k+1).

Basis: for k=0, the L.I. is trivially true.

Update: for all odd i, add to a[j] in that block the value a[i 2^k -1]

Final: when 2^k=n, the problem is solved.
</pre>

In [2]:


def partial_sum_block(a,k):
    n = 2**k
    
    def gpu_kernel_aux(a,i,j):
        a[i] += a[j]
    
    def gpu_kernel(a,i,j):
        # launch the kernels in parallel
        for t in range(i,j):
            gpu_kernel_aux(a,t,i-1)
    
    # assert LI
    m = 1
    for i in range(1,k+1):
        print(f'phase: {i}')
        #launch the kernels in sequence
        
        for s in range(1,n//m,2):
            gpu_kernel(a,s*m,(s+1)*m)

        m *= 2
        # assert LI
    # LI+termination = Goal
        
    

def test_partial_sum_cpu(k):
    n = 2**k
    a = np.random.randint(0,n,n)
    print(a)
    partial_sum_block(a,k)
    print(a)
    assert (sum(a)!=a[-1]), "checking that one value is correct"
    
test_partial_sum_cpu(5)



[22 12 31 26  1 15 16  4  4 14  2 12 26 20  9 12 23  3 31 19 23  0  7  9
 24 11 31  9 12 29 13  3]
phase: 1
phase: 2
phase: 3
phase: 4
phase: 5
[ 22  34  65  91  92 107 123 127 131 145 147 159 185 205 214 226 249 252
 283 302 325 325 332 341 365 376 407 416 428 457 470 473]


### Exercise: The chain method

Another way about this is to maintain as a loop invaiant that there are chains of partial sums with skip $2^k$, beginning at $2^k==n/2$ and working down to $2^k=0$. 

For the first time through the loop, what is established in parallel is,


In [6]:
def example_chain(a):
    n = len(a)
    for i in range(0,n//2):
        a[i] += a[i+n//2]

Then the chain skip is halved to $2^{k-1}$ and the loop invariant is restablished by merging chains starting at $i$ and $i+2^{k-1}$. These two chains fit together bisecting each other, so with a simple add (done in parallel on each of the chains) the chains are updated.

__Note:__ this has to be done in two steps, a read of values, and once done, a write of values.
